In [ ]:
# use image: dimension_reduction_benchmark

In [ ]:
import time 
from numpy import * 
import numpy as np
from scipy.sparse.linalg import eigs
from scipy.sparse import *  
import anndata 
import pandas as pd
import scanpy as sc 
import os
rd = np.random.RandomState(888) 
import sys
import warnings
warnings.filterwarnings("ignore") 
import scib_metrics
from sklearn.metrics import adjusted_rand_score, silhouette_score, adjusted_mutual_info_score
import snapatac2 as snap    
import episcanpy as epi 
import scipy as sp 

In [2]:
 
path = '/data/work/test_data'
files = ['10x_Brain5k.h5ad','10x_PBMC10k.h5ad','Buenrostro_2018.h5ad','Chen_NBT_2019.h5ad','GSE194122_subset.h5ad','Ma_Cell_2020.h5ad','Trevino_Cell_2021.h5ad','Yao_Nature_2021.h5ad', 'Zemke_2023_human_subset.h5ad', 'Zemke_2023_mouse.h5ad']
 
methods_all = ['SnapATAC2','SnapATAC2 (jaccard)','SnapATAC2 (svd)','epiScanpy']
 
dir_out = '/data/work/embedding_results/'
for method in methods_all:
    for dataName in files:
            print(dataName)
            adata = anndata.read(path+'/'+dataName)   
            if method == 'SnapATAC2':                     
                _, evecs = snap.tl.spectral(adata, features=None, inplace=False, distance_metric="cosine")
                np.savetxt(dir_out + method + dataName + "reduced_dim.tsv", evecs, delimiter="\t")
            elif method == 'SnapATAC2 (jaccard)':
                _, evecs = snap.tl.spectral(adata, features=None, inplace=False, distance_metric="jaccard")
                np.savetxt(dir_out + method + dataName + "reduced_dim.tsv", evecs, delimiter="\t")
            elif method == 'SnapATAC2 (svd)':
                feature_weights = snap.tl._embedding.idf(adata)
                X = adata.X @ sp.sparse.diags(feature_weights)
                s = 1 / np.sqrt(np.ravel(sp.sparse.csr_matrix.power(X, 2).sum(axis = 1)))
                X = sp.sparse.diags(s) @ X
                D = np.ravel(X @ X.sum(axis = 0).T)
                X = sp.sparse.diags(1 / np.sqrt(D)) @ X
                evecs,evals,_ = sp.sparse.linalg.svds(X, k=30)
                ix = evals.argsort()[::-1]
                evals = evals[ix]
                evecs = evecs[:, ix]
                idx = [i for i in range(evals.shape[0]) if evals[i] > 0]
                evals = evals[idx]
                evecs = evecs[:, idx] * evals
                np.savetxt(dir_out + method + dataName + "reduced_dim.tsv", evecs, delimiter="\t")
            elif method == 'epiScanpy':    
                def find_elbow(x):
                    n = len(x)
                    marks = []
                    saturation = 0.01
                    accum_gap = 0
                    for i in range(1, n):
                        gap = x[i-1] - x[i]
                        accum_gap += gap
                        if gap > saturation * accum_gap:
                            marks.append(i)
                    return min(n - 1, max(marks) + 1)
                data = adata
                data.X.data = data.X.data.astype(np.float64)
                epi.pp.normalize_per_cell(data)
                epi.pp.log1p(data)
                epi.pp.pca(data, n_comps=30)
                # sc.pl.pca_variance_ratio(data, log=True, save=".png")
                n_pc = find_elbow(data.uns['pca']['variance'])
                # print(f"Elbow point: {n_pc}")
                embedding = data.obsm['X_pca'][:, :n_pc]
                np.savetxt(dir_out + method + dataName + "reduced_dim.tsv", embedding, delimiter="\t") 
 
 

Zemke_2023_mouse.h5ad


In [3]:
import gc
gc.collect()

54